In [ ]:
import geopandas
import geoviews as gv
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
import panel as pn
import stmtools
import xarray as xr

from dask.distributed import Client
from holoviews import opts
from holoviews import streams

In [ ]:
# hv configurations
hv.extension('bokeh')
opts.defaults(opts.Points(tools=['box_select', 'lasso_select']))
gv.output(dpi=120)

# Visualize STM & contextual data

In [2]:
# input data paths
DATA_DIR = '/Users/fnattino/Projects/MobyLe/Data/spider/Public/demo_mobyle/data'

# Space-time matrix data
DATA_STEM = 'full-pixel_psi_amsterdam_tsx_asc_t116_v4_ampl_std_H_c16643'
CSV_STM_PATH = f'{DATA_DIR}/depsi_products/{DATA_STEM}.csv.part'
ZARR_STM_PATH = f'{DATA_DIR}/depsi_products/{DATA_STEM}.zarr'

# time-dependent variable: total precipitation
TP_DATA_PATH = f'{DATA_DIR}/ERA5-land-monthly_2015-2023_NL.nc'
# space-dependent variable: BAG dataset for AMS
BAG_DATA_PATH = f'{DATA_DIR}/bag_light_AMS_WGS84.gpkg'

## 1. Setup Dask cluster

In [3]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:58247,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:58258,Total threads: 2
Dashboard: http://127.0.0.1:58260/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:58250,


## 2. Convert STM data format: CSV -> Zarr

In [4]:
# load CSV file as Xarray Dataset
stm = stmtools.from_csv(CSV_STM_PATH, output_chunksize={'space': 25_000, 'time': -1})
stm = stm.persist()

In [5]:
# write dataset as Zarr store
stm.to_zarr(ZARR_STM_PATH, mode='w')

/var/folders/t6/r2gjczqj7bb8798wr4g1p87m0000gn/T/ipykernel_50061/3335913852.py:2: SerializationWarning: variable None has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  stm.to_zarr(ZARR_STM_PATH, mode='w')
/var/folders/t6/r2gjczqj7bb8798wr4g1p87m0000gn/T/ipykernel_50061/3335913852.py:2: SerializationWarning: variable None has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  stm.to_zarr(ZARR_STM_PATH, mode='w')


## 3. STM and contextual data

In [6]:
# STM dataset, space-time dependent
stm = xr.open_zarr(ZARR_STM_PATH)

In [7]:
# Total precipitation, (space-)time dependent
ds = xr.open_dataset(TP_DATA_PATH)
tp = ds['tp'].sel(
    latitude=stm['lat'].mean(), 
    longitude=stm['lon'].mean(),
    time=stm.time, 
    expver=1,
    method='nearest',
)
tp = tp.assign_coords(time=stm.time)

In [8]:
# BAG dataset, space dependent
bbox = stm.lon.min(), stm.lat.min(), stm.lon.max(), stm.lat.max()
bag = geopandas.read_file(BAG_DATA_PATH, bbox=bbox)

## 4. Visualizing the datasets

Scatter plot on base map for the STM data points:

In [9]:
# create points plot
xy = stm[['lat', 'lon']].to_dataframe()
points = xy.hvplot.points(
    'lon', 
    'lat', 
    geo=True, 
    color='red', 
    tiles='ESRI', 
)

Interactive line plot for the deformation of each point (deformation vs time):

In [31]:
TOO_MANY_POINTS = 10
VARIABLE = 'deformation'

# create stream for a selection of points
selection = streams.Selection1D(source=points)

def plot_variable(index):
    """ Plot STM variable vs time for a sub-set of points. """
    if not index or len(index) > TOO_MANY_POINTS:
        # for no or too many points, plot point 0
        return plot_variable([0])
    else:
        # 
        lines = [
            stm.isel(space=i).hvplot(x='time', y=VARIABLE)
            for i in index
        ]
        return hv.Overlay(lines)

# create interactive variable plot
deformation = hv.DynamicMap(plot_variable, streams=[selection])

Time-dependent contextual data: line plot for the total precipitation vs time.

In [29]:
precipitation = tp.hvplot.line(x='time', y='tp')

Space-dependent contextual data: building footprints

In [28]:
buildings = gv.Polygons(bag, vdims=[('bouwjaar', 'Year Built')])

Setting some parameters, and put it all together:

In [37]:
kwargs = dict(frame_width=500, frame_height=300)
points = points.opts(**kwargs)
buildings = buildings.opts(**kwargs, tools=['hover'])
deformation = deformation.opts(**kwargs)
precipitation = precipitation.opts(**kwargs)

plot = (points + buildings + deformation + precipitation).cols(2)

Display it in notebook:

In [38]:
plot

:Layout
   .Overlay.I    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [lon,lat]
   .Polygons.I   :Polygons   [Longitude,Latitude]   (bouwjaar)
   .DynamicMap.I :DynamicMap   []
      :Overlay
         .Curve.I :Curve   [time]   (deformation)
   .Curve.I      :Curve   [time]   (tp)

Or run it independently with a dedicated `panel` server:

In [39]:
pn.panel(plot).show()

Launching server at http://localhost:59063
